In [1]:
# importing necessary modules
import random
import csv
import pandas as pd
import numpy as np

In [2]:
# initializing global variables
class gv:
  startNum = 100 #initial number of skin cells (50-1000)
  result = 0 # 1 - bacteria won, 0 - immunity won
  numOfRounds = 0 # number of rounds till the end of the simulation

  # Percentages are represented as a fraction from 0 to 1 
  # Chances are represented as a number from 0 to 100
  
  # skin cells characteristics
  gVScRatio = 0.2 # proportion of guards to civilians (0.05-0.5 (5-50%))
  lifespanSkinCells = 2 # number of rounds untill skin cell will naturaly die (1-3 rounds)
  civiliansProductivity = 2 # units of food that civilians produce each round (1-5 units)
  guardsProductivity = 1 # number of bacteria that 1 guard kill during 1 round (1-3 bacteria)
  armyPriority = 0.50 # proportion of food given to the army each round (0.1-0.9 (10-90%))

  # bacteria characteristics
  bacteriaStartPercent = 0.3 # proportion of bacteria to skin cells at the begining of simulation (0.2-1 (20-100%))
  takingFoodEffectivity = 0.8 # proportion of food that bacteria take every round (0.1-0.9 (10-90%))
  lifespanBacteria = 5 # number of rounds untill bacterium will naturaly die (0-2 rounds)
  chanceOfKillingGuard = 30 # chance that 1 bacterium will kill 1 guard during 1 round (1-50%)
  chanceOfKillingCivilian = 30 # chance that 1 bacterium will kill 1 civilian during 1 round (1-100%)

  # lists that contain Cell objects
  guardsSet = []
  civiliansSet = []
  bacteriaSet = []

# class that defines every type of cell (all 3 of them) 
class Cell:
  def __init__(self, age, food):
    self.age = age
    self.food = food


In [3]:
# function that gives to initial variables random values within a given limits
def randomSimulation():
  gv.startNum = random.randint(50,1000)

  # skin cells params
  gv.gVScRatio = random.randint(5,50)/100 
  gv.lifespanSkinCells = random.randint(1,3) 
  gv.civiliansProductivity = random.randint(1,5) 
  gv.guardsProductivity = random.randint(1,3) 
  gv.armyPriority = random.randint(10,90)/100 

  # bacteria params
  gv.bacteriaStartPercent = random.randint(20,100)/100 
  gv.takingFoodEffectivity = random.randint(10,90)/100 
  gv.lifespanBacteria = random.randint(0,2) 
  gv.chanceOfKillingGuard = random.randint(1,50) 
  gv.chanceOfKillingCivilian = random.randint(1,100) 

In [4]:
# function that is used at the begining of the simulation to create lists of every cell type
def spawnSet():
  for x in range(int(gv.startNum*(1-gv.gVScRatio))):
    gv.civiliansSet.append(Cell(random.randint(0,gv.lifespanSkinCells), random.randint(0,2)))
  for x in range(int(gv.startNum*gv.gVScRatio)):
    gv.guardsSet.append(Cell(random.randint(0,gv.lifespanSkinCells), random.randint(0,2)))
  for x in range(int(gv.startNum*gv.bacteriaStartPercent)):
    gv.bacteriaSet.append(Cell(random.randint(0,gv.lifespanBacteria), random.randint(0,2)))

# function runs 1 round of the simulation using functions that represent every step of it
def runRound():
  divisionOfResources()
  war()
  extinction()

# civilian cells create a certain amount of food 
# and then it is divided among skin cells and bacteria, according to parameters of the simulation
def divisionOfResources():
  # producing food for all the cells
  globalFood = int(len(gv.civiliansSet)*gv.civiliansProductivity*random.random()) 
  # feeding civilians
  for civil in gv.civiliansSet: 
    civil.food +=1
    globalFood -=1
  # feeding bacteria
  bacteriasFood = int(globalFood*gv.takingFoodEffectivity) 
  globalFood -= bacteriasFood
  globalFood += feedCell(gv.bacteriaSet, bacteriasFood) #if there are no sells of this tipe we return food back
  # feeding guards
  guardsFood = int(globalFood*gv.armyPriority) 
  globalFood -= guardsFood
  globalFood += feedCell(gv.guardsSet, guardsFood) #if there are no el of this tipe we return food back
  # feeding civilians again
  feedCell(gv.civiliansSet, globalFood)  

# function that helps distribute food for each cell of the list
def feedCell(cellD, food): 
   while(food>0 and cellD != []): 
    for cell in cellD:
      if(food<=0):
        break
      cell.food +=1
      food -=1
   return food

# guards kill bacteria and bacteria kill skin cells
def war():
  # bacteria multiply
  for bacteria in gv.bacteriaSet: 
    if bacteria.food > 1:
      gv.bacteriaSet.append(Cell(0,0))
      bacteria.food -=1
  # guards kill bacteria
  del gv.bacteriaSet[0:len(gv.guardsSet)*gv.guardsProductivity] 
  # bacterias kill skin cells
  for bacteria in gv.bacteriaSet: 
    if random.randint(0,100) <= gv.chanceOfKillingGuard:
      del gv.guardsSet[0:1]
    if random.randint(0,100) <= gv.chanceOfKillingCivilian:
      del gv.civiliansSet[0:1]

# cells die if they haven't got any food or their life span expired
# survivors multiply if they get more than 1 unit of food.
def extinction():  
  cellDeath(gv.bacteriaSet, gv.lifespanBacteria)
  cellDeath(gv.civiliansSet, gv.lifespanSkinCells)
  cellDeath(gv.guardsSet, gv.lifespanSkinCells)

def cellDeath(cellD, lifeSpan):  
  survivors = []
  for cell in cellD:
    cell.age +=1
    if cell.food >= 1 and cell.age <= lifeSpan:
      survivors.append(cell)
  for c in survivors:
      if c.food >1:
        survivors.append(Cell(0,0))
        c.food -=1
      c.food -=1
  cellD = survivors      

In [5]:
# before the begining of the simulation we delete all the data of the last one 
# than we create sets of cells
# than we run rounds untill there are any bacteria or guards left
# we save final result and number of rounds in the global variables
def beginSim():
  del gv.civiliansSet[0:len(gv.civiliansSet)]
  del gv.guardsSet[0:len(gv.guardsSet)]
  del gv.bacteriaSet[0:len(gv.bacteriaSet)]
  spawnSet()
  numOfRounds = 0
  while len(gv.bacteriaSet) > 0 and len(gv.guardsSet) > 0:
   runRound()
   numOfRounds+=1
  if (len(gv.bacteriaSet) > 0):
   gv.result = 1
  else:
   gv.result = 0
  gv.numOfRounds = numOfRounds

In [6]:
# link to the database
db = 'data/SimulatorDatabase.csv'

In [ ]:
# !!! using this for second time will DELETE ALL THE DATA from database !!!
# creating empty database

def createEmptyDB():
    with open(db,mode = "w", newline='') as f: #creatind database
        dbWriter = csv.writer(f, delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        dbWriter.writerow(['startNum', 'gVScRatio', 'lifespanSkinCells', 'civiliansProductivity', 
        'guardsProductivity', 'armyPriority', 'bacteriaStartPercent', 'takingFoodEffectivity',
        'lifespanBacteria', 'chanceOfKillingGuard', 'chanceOfKillingCivilian', 'numOfRounds', 'result'])

In [8]:
# function appends database with the data of the last symulation
def appendDB(): 
  dList = [gv.startNum, gv.gVScRatio, gv.lifespanSkinCells, gv.civiliansProductivity, gv.guardsProductivity, 
           gv.armyPriority, gv.bacteriaStartPercent, gv.takingFoodEffectivity, gv.lifespanBacteria, gv.chanceOfKillingGuard, 
           gv.chanceOfKillingCivilian, gv.numOfRounds, gv.result]
  with open(db, mode = "a", newline='') as f:
    dbWriter = csv.writer(f, delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    dbWriter.writerow(dList)
    f.close()

In [13]:
# run random simulations n times and append it's data to database 
def runNtimes(n):
  for i in range(n): 
   randomSimulation()
   beginSim()
   appendDB()

In [10]:
# this function lets user to run a simulation with certain parameters
def certain_simulation(startNumN, gVScRatioN, lifespanSkinCellsN, civiliansProductivityN, guardsProductivityN, 
                       armyPriorityN, bacteriaStartPercentN, takingFoodEffectivityN, lifespanBacteriaN, chanceOfKillingGuardN, 
                       chanceOfKillingCivilianN):
  gv.startNum = startNumN

  #skin cells
  gv.gVScRatio = gVScRatioN 
  gv.lifespanSkinCells = lifespanSkinCellsN 
  gv.civiliansProductivity = civiliansProductivityN 
  gv.guardsProductivity = guardsProductivityN 
  gv.armyPriority = armyPriorityN 

  #bacteria
  gv.bacteriaStartPercent = bacteriaStartPercentN 
  gv.takingFoodEffectivity = takingFoodEffectivityN 
  gv.lifespanBacteria = lifespanBacteriaN 
  gv.chanceOfKillingGuard = chanceOfKillingGuardN
  gv.chanceOfKillingCivilian = chanceOfKillingCivilianN 

  del gv.civiliansSet[0:len(gv.civiliansSet)]
  del gv.guardsSet[0:len(gv.guardsSet)]
  del gv.bacteriaSet[0:len(gv.bacteriaSet)]
  spawnSet()
  while len(gv.bacteriaSet) > 0 and len(gv.guardsSet) > 0:
   runRound()
  if (len(gv.bacteriaSet) > 0):
   return 1
  else:
   return 0